In [1]:
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding
import matplotlib.pyplot as plt
from tqdm import tqdm
tqdm.pandas()
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional
from keras.datasets import imdb
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Bidirectional, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tqdm import tqdm
from sklearn.model_selection import ParameterGrid
from keras import layers
from keras.layers import Flatten, Dropout
from tensorflow.keras.models import load_model
from gensim.models import Word2Vec

In [2]:
############################################################################################################################

In [2]:
data = pd.read_excel('File_camaleonti.xlsx')

In [3]:
data.head()

,Testo
0,I camaleonti si nutrono principalmente di inse...
1,Alcune specie più grandi possono cacciare picc...
2,La loro lingua appiccicosa e rapida è l'arma p...
3,Le prede vengono afferrate con un movimento fu...
4,Questi rettili cacciano principalmente durante...


In [4]:
data.shape

(700, 1)

# Preprocessing
Notiamo che visto che noi vogliamo un algoritmo che sappia parlare, vogliamo tenre un testo più simile all'italiano possibile

In [5]:
# Usiamo le espessioni regolari per eliminari tutti i carattere non alfa-numerici
# \d tiene anche i caratteri numerici
data['Testo_Pulito'] = data.Testo.str.lower().str.replace('[^\w\s\d]',' ', regex=True)
data['Testo_Pulito'] = data['Testo_Pulito'].str.split()
# Leviamo tutte le maiuscole
data['Testo'] = data['Testo'].str.lower()
data['Testo'] = data.Testo.str.split()

In [6]:
from nltk.corpus import stopwords

# Rimuoviamo le stopwords e ricostruiamo il testo
stop = stopwords.words('italian')
stop.extend(['good', 'bad', 'dont', 'many', 'excellent', 'would', 'perfect', 'even', 'great','nice', 'amazing'])
data['Testo_Pulito'] = data['Testo_Pulito'].progress_apply(lambda x: ' '.join([item for item in x if item not in stop]))
data['Testo_Pulito'] = data['Testo_Pulito'].str.split()

100%|█████████████████████████████████████████████████████████████████████████████| 700/700 [00:00<00:00, 30368.36it/s]


In [7]:
from gensim.models.phrases import Phrases

# Bigrammi
bigram_model = Phrases(data['Testo_Pulito'], min_count=5, threshold=0.3)
data['Testo_Pulito'] = data['Testo_Pulito'].progress_apply(lambda x: bigram_model[bigram_model[x]])

100%|█████████████████████████████████████████████████████████████████████████████| 700/700 [00:00<00:00, 40481.09it/s]


In [8]:
data.head()

,Testo,Testo_Pulito
0,"[i, camaleonti, si, nutrono, principalmente, d...","[camaleonti, nutrono, principalmente, insetti,..."
1,"[alcune, specie, più, grandi, possono, cacciar...","[alcune_specie, grandi, possono, cacciare, pic..."
2,"[la, loro, lingua, appiccicosa, e, rapida, è, ...","[lingua_appiccicosa, rapida, arma, principale,..."
3,"[le, prede, vengono, afferrate, con, un, movim...","[prede, vengono, afferrate, movimento, fulmine..."
4,"[questi, rettili, cacciano, principalmente, du...","[rettili, cacciano, principalmente, durante, g..."


# LSTM

I questa parte andremo a trovare i migliori iperparametri e poi ad allenare il nostrom odello per ottenre una frase generata

# Param Grid
Creaiamo una funzone di supporto da poter inserire ne l nostro param-grid cambiando i parametri ogni volta.

In [9]:
max_sequence_length = 15  # E' a lunghezza massima della frase che vogliamo generare
### Creaiamo gli oggetti hce ci servono per allenare l'algoritmo:

In [9]:
testo_param = data.Testo.head(50)

In [10]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(testo_param) 
sequences = tokenizer.texts_to_sequences(testo_param)  

X, y = [], []  
for sequence in sequences: 
    for i in range(1, len(sequence)):
        X.append(sequence[:i])
        y.append(sequence[i])

X = pad_sequences(X, padding='post', maxlen=max_sequence_length - 1) 
y = np.array(y)
vocab_size = len(tokenizer.word_index) + 1

In [11]:
### Creazione della funzione di supporto:
def support_function(nodi_1,dr1,nodi_2,dr2,nodi_3,dr3,active,optimize,n_batch_size):
    model = Sequential()
    model.add(Embedding(vocab_size, 100, 
                        input_length=max_sequence_length - 1))
    model.add(Bidirectional(LSTM(nodi_1, return_sequences=True)))
    model.add(Dropout(dr1))
    model.add(Bidirectional(LSTM(nodi_2, return_sequences=True)))
    model.add(Dropout(dr2))
    model.add(Bidirectional(LSTM(nodi_3)))
    model.add(Dropout(dr3))
    model.add(Dense(vocab_size, activation=active))
    model.compile(loss='sparse_categorical_crossentropy',
                        optimizer=optimize, metrics=['accuracy'])
    
    model.fit(X, y, epochs=50, batch_size=n_batch_size, verbose = 0)
    
    lista_loss = model.evaluate(X,y) ### riporta una lista col valore della loss e della metrica utilizzata (in questo caso l'accuracy)
    
    return lista_loss

In [33]:
### Nodi per LSTM:
nodi_1 = [80, 100]
nodi_2 = [80, 100]
nodi_3 = [80, 100]

### drop out:
dr1 = [0.1,0.2,0.3]
dr2 = [0.1,0.2,0.3]
dr3 = [0.1,0.2,0.3]

### Tipologia di funzione di attivazione:
activation = ['relu', 'softmax']

### Tipologia di funzione di ottimizzazione per la funzione di perdita:
optimizer = ['adam', 'rmsprop']

### Batch size:
batch_size= [32, 64]

risultati = {
    'nodi_1': [],
    'dr1':[],
    'nodi_2': [],
    'dr2':[],
    'nodi_3':[],
    'dr3':[],
    'activation' : [],
    'optimizer' : [],
    'batch_size' : [],
    'loss':[],
    'accuracy':[] 
}


if 1 == 1:     ### Codice per far partire il tqdm
    pbar = tqdm(total=(len(lstm_units)*len(activation)*len(optimizer)*len(epochs)*len(batch_size)))

    for n1 in nodi_1:  ### 'nodi_lstm' questo assume i valori della lista
        for d1 in dr1:
            for n2 in nodi_2:
                for d2 in dr2:
                    for n3 in nodi_3:
                        for d3 in dr3:
                            for active in activation:
                                for optimize in optimizer:
                                    for n_batch_size in batch_size:
                        
                                        cv = support_function(n1,d1,n2,d2,n3,d3,active,optimize,n_batch_size)
                                        los = cv[0]
                                        accur = cv[1]
                        
                                        risultati['nodi_1'].append(n1)
                                        risultati['dr1'].append(d1)
                                        risultati['nodi_2'].append(n2)
                                        risultati['dr2'].append(d2)
                                        risultati['nodi_3'].append(n3)
                                        risultati['dr3'].append(d3)
                                        risultati['activation'].append(active)
                                        risultati['optimizer'].append(optimize)
                                        risultati['batch_size'].append(n_batch_size)

                                        risultati['loss'].append(los)
                                        risultati['accuracy'].append(accur)
                    
                        
                        
                        
                                        pbar.update(1)
    pbar.close()


  0%|                                                                                           | 0/16 [03:32<?, ?it/s]


23/23 [==============================] - 3s 16ms/step - loss: 5.4889 - accuracy: 0.0000e+00



  6%|█████▏                                                                             | 1/16 [01:16<19:12, 76.82s/it]

23/23 [==============================] - 4s 42ms/step - loss: 5.4889 - accuracy: 0.0000e+00



 12%|██████████▎                                                                       | 2/16 [03:20<24:18, 104.15s/it]

...

23/23 [==============================] - 5s 115ms/step - loss: 2.3615 - accuracy: 0.4050



 94%|██████████████████████████████████████████████████████████████████████████     | 15/16 [1:04:04<05:20, 320.45s/it]

23/23 [==============================] - 5s 112ms/step - loss: 2.9272 - accuracy: 0.2751



100%|███████████████████████████████████████████████████████████████████████████████| 16/16 [1:07:27<00:00, 285.11s/it]

KeyboardInterrupt: 

In [34]:
df_results = pd.DataFrame(data = risultati)
pd.DataFrame(df_results).to_csv('param_grid.csv', index=False)
df_results.head(5)

,nodi_1,dr1,nodi_2,dr2,nodi_3,dr3,activation,optimizer,batch_size,loss,accuracy
0,80,0.1,80,0.1,80,0.1,relu,adam,32,5.488938,0.000000
1,80,0.1,80,0.1,80,0.1,relu,adam,64,5.488938,0.000000
2,80,0.1,80,0.1,80,0.1,relu,rmsprop,32,5.488938,0.000000
3,80,0.1,80,0.1,80,0.1,relu,rmsprop,64,5.488938,0.000000
4,80,0.1,80,0.1,80,0.1,softmax,adam,32,1.437046,0.666201


In [35]:
best_accuracy = df_results.sort_values(by ='accuracy' ,axis=0, ascending=False, ignore_index=True)
best_parameters_accuracy = best_accuracy.loc[0]
best_parameters_accuracy

nodi_1              80
dr1                0.1
nodi_2              80
dr2                0.1
nodi_3              80
dr3                0.1
activation     softmax
optimizer         adam
batch_size          32
loss          1.437046
accuracy      0.666201
Name: 0, dtype: object

In [36]:
best_loss = df_results.sort_values(by ='loss' ,axis=0, ascending=True, ignore_index=True)
best_parameters_loss = best_loss.loc[0]
best_parameters_loss

nodi_1              80
dr1                0.1
nodi_2              80
dr2                0.1
nodi_3              80
dr3                0.1
activation     softmax
optimizer         adam
batch_size          32
loss          1.437046
accuracy      0.666201
Name: 0, dtype: object

# Creazione LSTM
in questa parte andreamo ad allenrare il nostro algoritmo con la migliore configurazione trovata al punto precedente

In [10]:
def preprocess_data(texts, max_sequence_length):
    tokenizer = Tokenizer()  ### è ciò che viene utilizzato per andare a tokenizzare i testi (in liste di interi)
    tokenizer.fit_on_texts(texts)  ### questo è il comando col quale andiamo a creare le liste di interi
    sequences = tokenizer.texts_to_sequences(texts)  ### il rigo prima allena il modello "tokenizer" e questo trasforma effettivamente le parole in numeri

    X, y = [], []  ### vengono crate delle liste per allanere LSTM. 
    for sequence in sequences:  ### X è la lista con la quale vienve predetta Y (es. per la frase "il sole splende", si avranno rispettivamente X e Y: ("Il", "sole") e ("Il", "sole", "splende"))
        for i in range(1, len(sequence)):
            X.append(sequence[:i])
            y.append(sequence[i])

    X = pad_sequences(X, padding='post', maxlen=max_sequence_length - 1) ### con questo codice si allineano le sequenze di parole nelle X alla lunghezza massima da noi specificata -1
    y = np.array(y) ### Y viene traformata in array                      ### con padding='post' tutte le sequenze che non raggiungono la lunghezza massima di almeno quella specificat da noi verranno completate con 0.
    vocab_size = len(tokenizer.word_index) + 1             ### viene calcolato il valore del vocabolario, ovvero il totale di numeri unici (quindi di parole uniche) + 1 che sarebbe lo 0 nel caso non ci fossero abbastanza parole

    return X, y, vocab_size, tokenizer

In [11]:
def build_lstm_model(vocab_size, max_sequence_length):
    model = Sequential()  ### viene creato l'oggetto "Sequential" ovvero una rete feedforward dove l'output di uno strato viene utilizzato come input dello strato successivo
    model.add(Embedding(vocab_size, 100, input_length=max_sequence_length - 1))  ### viene aggiunto lo strato di embedding che ha come parametri vocab_size, un intero che è la lunghezza massima degli embeddings e input_length=max_sequence_length - 1 definisce quanti nodi di input servano per iniziare (ovviamente tatni quanti la lunghezza massima di parole di una frase -1 come nel preprocess)
    model.add(Bidirectional(LSTM(80, return_sequences=True)))  ### viene aggiunto un layer di 64 nodi di tipo LSTM ovvero un partiolare tipo di rete ricorsiva di tipo bidirezionale
    model.add(Dropout(0.1))  # Aggiunto dropout per la regolarizzazione
    model.add(Bidirectional(LSTM(80, return_sequences=True)))
    model.add(Dropout(0.1))  # Aggiunto dropout per la regolarizzazione
    model.add(Bidirectional(LSTM(80)))
    model.add(Dropout(0.1))
    model.add(Dense(vocab_size, activation='softmax'))  ### questo è un layere di output ovvero uno strato dove si utilizza una funzione di attivaazione per calcolare la probabiltà maggiore delle parole del vocabolario di essere l'output. questo layer è completamente connesso
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [12]:
def train_model(model, X, y, epochs, batch_size):
    model.fit(X, y, epochs=epochs, batch_size=batch_size)   ### il batch è il numero di esempi di partenza che vengono presi ad ogni epoca. Piò batch si usano più l'allenamento sarà veloce, ma richiederà più memoria.

In [13]:
# Supponendo di avere il testo delle frasi in una lista chiamata 'texts'
 # Puoi impostare la lunghezza massima della sequenza in base alle tue esigenze
# X, y, vocab_size, tokenizer = preprocess_data(data.Testo_Pulito, max_sequence_length)
X, y, vocab_size, tokenizer = preprocess_data(data.Testo_Pulito, max_sequence_length)
model = build_lstm_model(vocab_size, max_sequence_length)

In [14]:
# Addestramento del modello
epochs=100
batch_size=32
train_model(model, X, y, epochs, batch_size)

Epoch 1/100
148/148 [==============================] - 15s 39ms/step - loss: 6.8894 - accuracy: 0.0161
Epoch 2/100
148/148 [==============================] - 6s 40ms/step - loss: 6.5791 - accuracy: 0.0195
Epoch 3/100
148/148 [==============================] - 6s 42ms/step - loss: 6.4149 - accuracy: 0.0208
Epoch 4/100
148/148 [==============================] - 6s 43ms/step - loss: 6.2495 - accuracy: 0.0195
Epoch 5/100
148/148 [==============================] - 7s 44ms/step - loss: 6.1041 - accuracy: 0.0255
Epoch 6/100
148/148 [==============================] - 7s 44ms/step - loss: 5.9568 - accuracy: 0.0261
Epoch 7/100
148/148 [==============================] - 7s 44ms/step - loss: 5.8540 - accuracy: 0.0286
Epoch 8/100
148/148 [==============================] - 7s 44ms/step - loss: 5.7219 - accuracy: 0.0293
Epoch 9/100
148/148 [==============================] - 8s 51ms/step - loss: 5.5798 - accuracy: 0.0363
Epoch 10/100
148/148 [==============================] - 8s 52ms/step - loss: 5.43

148/148 [==============================] - 10s 66ms/step - loss: 0.7192 - accuracy: 0.7889
Epoch 81/100
148/148 [==============================] - 9s 64ms/step - loss: 0.7167 - accuracy: 0.7921
Epoch 82/100
148/148 [==============================] - 9s 62ms/step - loss: 0.7099 - accuracy: 0.7902
Epoch 83/100
148/148 [==============================] - 9s 61ms/step - loss: 0.6939 - accuracy: 0.7947
Epoch 84/100
148/148 [==============================] - 9s 60ms/step - loss: 0.6785 - accuracy: 0.8000
Epoch 85/100
148/148 [==============================] - 9s 58ms/step - loss: 0.6938 - accuracy: 0.7949
Epoch 86/100
148/148 [==============================] - 9s 59ms/step - loss: 0.6867 - accuracy: 0.7934
Epoch 87/100
148/148 [==============================] - 10s 65ms/step - loss: 0.6810 - accuracy: 0.7968
Epoch 88/100
148/148 [==============================] - 9s 58ms/step - loss: 0.6326 - accuracy: 0.8082
Epoch 89/100
148/148 [==============================] - 9s 59ms/step - loss: 0.6393 

In [16]:
model.save("modello_pre_process.h5")

In [16]:
#model = load_model("modello_tre_layer.h5")

# Proviamo ad usare word2vec per lo strato di embedding

In [52]:
## Facciamo il word embedding con word2vec
sentences = data['Testo']

# Supponendo che 'sentences' sia una lista di liste di parole
model = Word2Vec(sentences, vector_size=100, window=10, min_count=1, sg=0) 
model.train(sentences, total_examples=len(sentences), epochs=10)


# Estrai il vocabolario e gli embeddings dal modello Word2Vec
vocab = model.wv.key_to_index
word_vectors = model.wv

# Dimensioni della matrice degli embeddings
vocab_size = len(vocab)
embedding_dim = word_vectors.vector_size

# Inizializza una matrice vuota per gli embeddings
embedding_matrix = np.zeros((vocab_size, embedding_dim))

# Riempire la matrice con gli embeddings Word2Vec
for word, index in vocab.items():
    if word in word_vectors:
        embedding_matrix[index] = word_vectors[word]



In [53]:
from keras.models import Sequential
from keras.layers import Embedding, Bidirectional, LSTM, Dense
embedding_layer = Embedding(input_dim=vocab_size, output_dim=100, 
                            input_length=max_sequence_length - 1, weights=[embedding_matrix], trainable=False)


def build_lstm_model2(vocab_size, max_sequence_length):
    model = Sequential()  ### viene creato l'oggetto "Sequential" ovvero una rete feedforward dove l'output di uno strato viene utilizzato come input dello strato successivo
    model.add(embedding_layer)
    model.add(Dropout(0.2))  # Aggiunto dropout per la regolarizzazione
    model.add(Bidirectional(LSTM(64)))
    model.add(Dropout(0.2))  # Aggiunto dropout per la regolarizzazione
    model.add(Dense(vocab_size, activation='softmax'))  ### questo è un layere di output ovvero uno strato dove si utilizza una funzione di attivaazione per calcolare la probabiltà maggiore delle parole del vocabolario di essere l'output. questo layer è completamente connesso
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

def train_model(model, X, y, epochs, batch_size):
    model.fit(X, y, epochs=epochs, batch_size=batch_size)

In [54]:
X, y, vocab_size, tokenizer = preprocess_data(data.Testo, max_sequence_length)
model = build_lstm_model2(vocab_size, max_sequence_length)

# Addestramento del modello
epochs=250
batch_size=64
train_model(model, X, y, epochs, batch_size)

Epoch 1/250
16/60 [=======>......................] - ETA: 1s - loss: 7.2275 - accuracy: 0.0117

KeyboardInterrupt: 

# Generazione della frase

In [17]:
def generate_sentence(model, tokenizer, start_words, max_sequence_length):
    start_words = start_words.split() ### Viene suddivisa la frase in una lista di parole
    
    # Converti le parole di inizio in una lista di sequenze di input
    start_sequences = tokenizer.texts_to_sequences(start_words)

    # Inizializza la frase con le parole di inizio
    sentence = start_words[:]

    while True:
        next_words = []   ### Viene iterato finchè non viene raggiunto il limite di 'max_sequence_length' oppure un punto

        for start_sequence in start_sequences:
            # Prevedi le probabilità di ogni parola nel vocabolario
            X = pad_sequences([start_sequence], padding='post', maxlen=max_sequence_length - 1)  ### Viene creato il set di allenamento per ogni parola predetta per predirre la successiva
            predictions = model.predict(X, verbose=0)   ### Viene predetta la parola successiva
            next_word_index = np.argmax(predictions)    ### Viene trovato il numero (token) della parola predetta

            # Converti l'indice della parola predetta in una parola reale
            next_word = tokenizer.index_word[next_word_index]

            # Aggiungi la parola predetta alla lista di parole successive
            next_words.append(next_word)

            # Aggiorna la sequenza di input con la nuova parola predetta
            start_sequence.append(next_word_index)
            #start_sequence = start_sequence[1:]    ### questo è uno slicing, infatti la parola [0] viene leiminata per tenere la lunghezza della lista sempre uguale.

        # Aggiungi le parole predette alla frase
        sentence.extend(next_words)

        # Interrompi il ciclo se viene generato il punto o la frase ha raggiunto la lunghezza massima
        if '.' in next_words or len(sentence) >= max_sequence_length:
            break

    return ' '.join(sentence)

In [20]:
# Utilizza la funzione generate_sentence per generare una frase
start_words = input('Parola inziale: ')
generated_sentence = generate_sentence(model, tokenizer, start_words, max_sequence_length)
print("Frase generata:", generated_sentence)

Parola inziale: camaleonti mangiano
Frase generata: camaleonti mangiano abitano pazienza zone fondamentali confine successo ambienti caccia diversi caccia dimostrando camaleonti versatilità opportunità
